In [3]:
import os
import seaborn as sns
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [4]:
def softmax(x, T = 1):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x / T) / np.sum(np.exp(x / T), axis=0)

In [5]:
cifar_resultsdir = '/vol/biomedic3/zl9518/ModelEvaluation/LDAM-DRW/cifar10results/'
cnn_pred = pd.read_csv(cifar_resultsdir + 'predictions_val.csv')

In [6]:
kcls = 10
targets_all = np.array(cnn_pred[['target_0', 'target_1', 'target_2', 'target_3', 'target_4', 'target_5', 'target_6', 'target_7', 'target_8', 'target_9']])
logit_all = np.array(cnn_pred[['logit_0', 'logit_1', 'logit_2', 'logit_3', 'logit_4', 'logit_5', 'logit_6', 'logit_7', 'logit_8', 'logit_9']])
# acc
target_class = np.argmax(targets_all, axis = 1)
pred_class = np.argmax(logit_all, axis = 1)
acc = np.sum(pred_class == target_class) / len(target_class)
prob = softmax(logit_all.transpose(), T = 1).transpose()
probmax = np.max(prob, axis = 1)
AC = np.mean(probmax)
print(acc)
print(AC)

mean_prob = []
targets_all = []
acc_cls = []
preds_class_all = []
preds_all = np.array(cnn_pred[['class_0', 'class_1', 'class_2', 'class_3', 'class_4', 'class_5', 'class_6', 'class_7', 'class_8', 'class_9']])
preds_all_argmax = np.argmax(preds_all, axis = 1)
preds_all_max = np.max(preds_all, axis = 1)
class_all = np.array(cnn_pred[['target_0', 'target_1', 'target_2', 'target_3', 'target_4', 'target_5', 'target_6', 'target_7', 'target_8', 'target_9']])
class_all_argmax = np.argmax(class_all, axis = 1)
for label in range(kcls):
    targets_y1 = np.where(preds_all_argmax==label)[0]
    preds_class = preds_all_max[targets_y1]
    
    preds_class_all = np.concatenate((preds_class_all, preds_class), axis=0)
    
    preds_realclass = class_all_argmax[targets_y1]
    targets_all = np.concatenate((targets_all, np.ones(len(preds_class)) * label), axis=0)
    acc_cls.append(np.sum(preds_realclass == label) / len(targets_y1))
    mean_prob.append(np.mean(preds_class))
print(mean_prob)
print(acc_cls)

0.7054
0.9106864566517481
[0.9225970566011684, 0.9465175189373298, 0.8975987029430894, 0.8778312979804334, 0.9152566284004128, 0.8755787677997529, 0.9290975124124513, 0.9130334730434783, 0.9070707217171718, 0.9059517615280899]
[0.5130111524163569, 0.667574931880109, 0.6723577235772358, 0.5485674353598882, 0.7966976264189887, 0.7688504326328801, 0.920881971465629, 0.9548494983277592, 0.9671717171717171, 0.9842696629213483]


In [7]:
# bring a test condition here.
acc_results = []
AC_results = []
DoC_results = []
CSDoC_results = []
cifar_resultsdir = '/vol/biomedic3/zl9518/ModelEvaluation/LDAM-DRW/cifar10results/'
corruptions = ['gaussian_noise', 'shot_noise' , 'impulse_noise', 'defocus_blur', 'glass_blur', 
               'motion_blur', 'zoom_blur', 'snow', 'frost', 'fog', 'brightness', 'contrast' , 'elastic_transform', 
               'pixelate', 'jpeg_compression', 'speckle_noise', 'gaussian_blur', 'spatter', 'saturate']

for cname in tqdm(corruptions):
    csvfilename = cifar_resultsdir + 'predictions_val_' + cname + '.csv'
    cnn_pred_all = pd.read_csv(csvfilename)
    for severity in range(5):
        cnn_pred = cnn_pred_all.iloc[severity * 10000:(severity + 1) * 10000, :]
        
        kcls = 10
        targets_all = np.array(cnn_pred[['target_0', 'target_1', 'target_2', 'target_3', 'target_4', 'target_5', 'target_6', 'target_7', 'target_8', 'target_9']])
        logit_all = np.array(cnn_pred[['logit_0', 'logit_1', 'logit_2', 'logit_3', 'logit_4', 'logit_5', 'logit_6', 'logit_7', 'logit_8', 'logit_9']])
        preds_all_argmax = np.argmax(logit_all, axis = 1)
        # acc
        target_class = np.argmax(targets_all, axis = 1)
        pred_class = np.argmax(logit_all, axis = 1)
        acc_t = np.sum(pred_class == target_class) / len(target_class)
        prob = softmax(logit_all.transpose(), T = 1).transpose()
        probmax = np.max(prob, axis = 1)
        acc_results.append(acc_t)
        AC_results.append(np.mean(probmax))
        DoC_results.append(acc-(AC-np.mean(probmax)))
        targets_all = []
        preds_class_all = []
        for label in range(kcls):
            preds_all = softmax(logit_all.transpose(), T = 1).transpose()
            preds_all_max = np.max(preds_all, axis = 1)
            targets_y1 = np.where(preds_all_argmax==label)[0]
            preds_class = preds_all_max[targets_y1]

            preds_class_all = np.concatenate((preds_class_all, preds_class - mean_prob[label] + acc_cls[label]), axis=0)
        CSDoC_results.append(np.mean(preds_class_all))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:12<00:00,  1.49it/s]


In [8]:
print('AC_results:')
print(np.mean(np.abs(np.array(acc_results)-np.array(AC_results))))
print('DoC_results:')
print(np.mean(np.abs(np.array(acc_results)-np.array(DoC_results))))
print('CSDoC_results:')
print(np.mean(np.abs(np.array(acc_results)-np.array(CSDoC_results))))

AC_results:
0.31302850098791185
DoC_results:
0.10774204433616379
CSDoC_results:
0.09364027001796721


In [9]:
print(DoC_results)

[0.6716073722530063, 0.6318219332256849, 0.5989617985116201, 0.5871380349866165, 0.5788520182162653, 0.6870734315750555, 0.6681572243257768, 0.6193758363166537, 0.6069041840553213, 0.5857611246509108, 0.6860774070717457, 0.6599561063075562, 0.6323169366145046, 0.5898560679621445, 0.5646759357742686, 0.7033293195049252, 0.6954346322141653, 0.6827864273978638, 0.6705762056526218, 0.6505191758191524, 0.6159253532778608, 0.6215837041210042, 0.6199667163655169, 0.6128458708757504, 0.6034150220897436, 0.689653862613752, 0.6775421683654855, 0.6688154415359313, 0.6664976643148346, 0.6619941818150136, 0.6738177585318718, 0.672227908459391, 0.6652355326478668, 0.6605590284030467, 0.6492553903199693, 0.6883557232202934, 0.6528962810703012, 0.661865103335706, 0.6575448702143369, 0.6393778643605073, 0.693069003645487, 0.6727643993887014, 0.6464146991517457, 0.6436038633902954, 0.6104579779564218, 0.703607992605194, 0.6929798829922625, 0.6838380643483326, 0.6738550245625128, 0.6449719083731476, 0.70

In [10]:
print(CSDoC_results)

[0.6927921957905866, 0.6679437190389814, 0.6542379931253861, 0.6515859839670718, 0.6545926304524582, 0.7060128864532011, 0.6931471591237388, 0.6616096964671422, 0.6562421314805531, 0.6542255258395552, 0.6747117040830463, 0.6474164651155273, 0.6295047405007017, 0.608729285345344, 0.6072372226841424, 0.6994448696284493, 0.6780258054466418, 0.6436471425777418, 0.610923604357552, 0.5626045354772516, 0.6273601034432565, 0.621012046288981, 0.6161015875716634, 0.6173577343923518, 0.6084102282754531, 0.6495024212610914, 0.6157985792527156, 0.5928716820502022, 0.5904289817788217, 0.5735827744269826, 0.634995672029316, 0.6203518722409332, 0.6025026986055491, 0.5864620869035163, 0.564112573036458, 0.6795100415432971, 0.6609545827740753, 0.6543770065675176, 0.6435338978853988, 0.63779810045447, 0.6940060523959187, 0.6704343303615514, 0.6466478845016163, 0.6462128702740685, 0.6235765851797247, 0.6960466390211781, 0.6726557224879288, 0.652106278271263, 0.6306965205378017, 0.5884530065304899, 0.70577